In [145]:
import pandas as pd
import import_ipynb
import h1_ENB_occ as h1 
import h2_ENB_gen as h2 


import pickle
import numpy as np
from scipy.sparse import csr_matrix
import nltk
nltk.download('wordnet')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import MWETokenizer
import string
import collections
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
from nltk import ngrams
import pandas as pd 
import re 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [146]:
def extract_list_issue():
     with open("Text/ENB_relationships.txt") as infile:
        k=0
        num = []
        for line in infile:
            x = line.replace('"','').split('\t')[:26]

            if(k > 0 and  508 < int(x[23]) and int(x[23]) < 510 ):

                num.append(int(x[23]))
            k +=1
        return list(set(num))

In [147]:
list_issue_num = extract_list_issue()

In [148]:
def extract_gen():
    frames1 = []
    frames2 = []
    s1 = []
    s2 = []
    for issue_number in list_issue_num:
        df1 = h1.extract_relationships_count(issue_number)
        df2 = h2.interactions(issue_number)
        frames1.append(df1[0])
        frames2.append(df2[0]) 
        s1.append(df1[1])
        s2.append(df2[1])
    return pd.concat(frames1), pd.concat(frames2), s1, s2


In [149]:
df_original, df_generated, so, sg = extract_gen()
number_original = len(df_original['issue'].to_numpy())
number_generated = len(df_generated['issue'].to_numpy())


Issue  509


In [150]:
import re
def clean_tp(sentence):
    """ Clean the sentence by removing special char."""
    s = re.sub('\n',' ',sentence)
    s = re.sub(r'\\',' ',s)
    s = s.replace("\r\n\s\s+"," ")
    s = s.replace("\r\n"," ")
    s = s.replace('\t','')
    s = s.replace("\s\s+"," ")
    s = s.replace("\\."," ")
    s = s.replace("\\r\\n"," ")
    p = re.compile(r'<.*?>')
    return p.sub('', s)

In [151]:
def tokenize_sentence(sentence, country):
    """Split the sentence in a way that the entities are together and will be able to be detected."""
    # Extract list entities
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]
    list_entities = [s.replace('class="textstory"','') for s in list_entities]
    
    tokens_entities = [l.split(' ') for l in list_entities]
    if(country):
        tokens_entities.append(['on','behalf','of'])
        tokens_entities.append(['for'])
        tokens_entities.append(['US','$'])
        tokens_entities.append(['speaking','for'])
    else :
        tokens_entities.append(['speaking','for','the'])
        tokens_entities.append(['speaking','for'])
        tokens_entities.append(['on','behalf','of','the'])
        tokens_entities.append(['spoke','with'])
        tokens_entities.append(['on','behalf','of'])
        tokens_entities.append(['for','the'])
        tokens_entities.append(['US','$'])
        
    tokens_entities.append(['supported','by'])
    tokens_entities.append(['opposed','by'])
    tokenizer1 = MWETokenizer(tokens_entities, separator=' ')
    tokenizer2 = MWETokenizer([['G-77','CHINA']], separator='/')
    tokenizer3 = MWETokenizer([['G-77/',' CHINA']], separator=' ')
    if(type(sentence) == list):
        line = sentence[0].replace(",","")
    else: 
        line = sentence.replace(",","")
    line_splited = word_tokenize(line)
    tokens = tokenizer1.tokenize(line_splited) 
    tokens = tokenizer2.tokenize(tokens) 
    tokens = tokenizer3.tokenize(tokens) 
    tokens = [clean_tp(token) for token in tokens]

    return tokens

In [152]:
def similarities(d_i,d_j):
    return np.dot(d_i.T,d_j)/(np.linalg.norm(d_i)*np.linalg.norm(d_j))

In [153]:
def define_df(df_original, so, df_generated, sg):
    se_orig = set(so[0])
    df_original['sentences'] = df_original['sentences'].apply(lambda x :"orig - "+x )
    se_orig = ["orig - "+x for x in se_orig]
    se_gen = set(sg[0])
    issues = df_original['issue'].drop_duplicates(keep = 'first')
    d_original = pd.DataFrame()
    d_generated= pd.DataFrame()
    #Split sentence
    s1 = [tokenize_sentence(x, True) for x in se_orig]
    s2 = [tokenize_sentence(x, True) for x in se_gen]
    d_original['sentences_splited'] = s1
    d_generated['sentences_splited'] = s2
    d_original['sentences'] = se_orig
    d_generated['sentences'] = se_gen
    sent_id_o = np.arange(len(se_orig))
    sent_id_g = max(sent_id_o) +1 +np.arange(len(se_gen))
    d_combines = pd.concat([d_original,d_generated])
    return d_combines, sent_id_o, sent_id_g

In [154]:
def find_word(description,word):
    if(description.count(word) > 0):
        return 1
    else:
        return 0

In [155]:
def find_n(list_description, word):
    #print([find_word(description, word) for description in list_description])
    return np.sum([find_word(description, word) for description in list_description])

In [156]:
def define_max_k(description):
    counter=collections.Counter(description)
    return counter.most_common(1)[0][1]

In [157]:
def count_(word,description,max_k):
    return description.count(word)/max_k    

In [158]:
def define_TFIDf_matrix(d_combines, sent_id_o, sent_id_g):

    N = max(sent_id_g) +1

    #Find all the words in all the documents
    sentences = d_combines['sentences_splited'].to_numpy().flatten()
    words = []
    for d in sentences:
        words += d
    words = set(words)
    #Data frame with words and their index
    d = {'word': list(words), 'index': np.arange(len(words))}
    df_words =  pd.DataFrame(data=d)

    indices = np.arange(N)
    d_combines['index'] = indices


    dictionary_doc =dict(zip(indices,d_combines['sentences_splited'].to_numpy()))
    dictionary_max_k =dict(zip(indices,d_combines.apply(lambda x: define_max_k(x['sentences_splited']),axis =1 )))
    df_words['n'] = [find_n(d_combines['sentences_splited'].to_numpy(), w) for w in df_words['word'].to_numpy()]

    df_words['IDF'] = df_words['n'].apply(lambda x: - np.log2(int(x)/N))

    for id_d in indices:
        df_words[str(id_d)] = [count_(w,dictionary_doc[id_d],dictionary_max_k[id_d]) for w in df_words['word'].to_numpy() ]

    df_words_TF = df_words.drop(['word', 'index','n','IDF'], axis=1)
    matrix_words_TF = df_words_TF.to_numpy()
    matrix_words_IDF = df_words['IDF'].to_numpy()
    marix_TFIDF = (matrix_words_TF.T * matrix_words_IDF).T
    return marix_TFIDF

In [159]:
def define_dict_sentences(d_combines, matrix_TFIDF, sent_id_o, sent_id_g):
    dict_sentence_idx = d_combines.set_index('sentences')['index'].to_dict()
    dict_idx_sentence = d_combines.set_index('index')['sentences'].to_dict()
    # Similarties with high Tf_DFI score
    keys = []
    values = []
    print(sent_id_g)
    for i in range(len(sent_id_g)):
        # Index of sentence in generated
        test_idx = sent_id_g[i]


        #Find sentence with best similarities
        test_v = matrix_TFIDF[:,test_idx ]
        similarities_list = [similarities(matrix_TFIDF[:,i],test_v) for i in sent_id_o ]
        test_s = np.argmax(similarities_list)
        """ print('Generated sentence : \n',dict_idx_sentence[test_idx], test_idx)
        print('Original sentence : \n',dict_idx_sentence[test_s], test_s)
        print('---------------------------------------------------------------------------------------------------------------------') """
        keys.append(test_idx)
        values.append(test_s)
    dictionary_sentences_id = dict(zip(keys, values))
    #Find the corresponding index of the sentence in the issue
    df_generated['se_index'] = df_generated['sentences'].apply(lambda x :dict_sentence_idx[x])
    df_original['se_index'] = df_original['sentences'].apply(lambda x :dict_sentence_idx[x])
    return dictionary_sentences_id, dict_sentence_idx, dict_idx_sentence, df_original, df_generated

In [160]:
def find_differences_issue(dictionary_sentences_id):
    frames = []
    num_o = 0
    num_g = 0
    for x in dictionary_sentences_id:
        index_g = x
        df_g = df_generated[df_generated.apply(lambda x: x['se_index'] == index_g, axis=1)]
        df_o = df_original[df_original.apply(lambda x: x['se_index'] == dictionary_sentences_id[index_g], axis=1)]
        df_g = df_g.drop(['behalf','support','agreement','opposition','criticism','se_index'], axis = 1)
        df_o = df_o.drop(['behalf','support','agreement','opposition','criticism','se_index'], axis = 1)
        num_o += len(df_o['cooperation'].to_numpy())
        num_g += len(df_g['cooperation'].to_numpy())
        #df_g = df_g.drop(['Country A','Country B','behalf','support','agreement','opposition','criticism','se_index'], axis = 1)
        #df_o = df_o.drop(['Country A','Country B','behalf','support','agreement','opposition','criticism','se_index'], axis = 1)
        d_concat = pd.concat([df_o,df_g])
        d_concat = d_concat.drop_duplicates(subset=['id_ca','id_cb','cooperation'], keep=False)
        frames.append(d_concat)

    return pd.concat(frames), num_o, num_g

In [161]:
df_combines, sent_id_o, sent_id_g =  define_df(df_original, so, df_generated, sg)
marix_TFIDF = define_TFIDf_matrix(df_combines, sent_id_o, sent_id_g)
print(marix_TFIDF.shape)
dictionary_sentences_id, dict_sentence_idx, dict_idx_sentence, df_original, df_generated = define_dict_sentences(df_combines, marix_TFIDF, sent_id_o, sent_id_g)
df_result, num_o, num_g = find_differences_issue(dictionary_sentences_id)


N : 16
(189, 16)
[11 12 13 14 15]


In [162]:
print('Original interactions : ', num_o)
print('Generated interactions : ',num_g)
print('Number of different interactions : ',len(df_result['issue'].to_numpy()))
print('Number of different interactions form generated : ', len(df_result[df_result.apply(lambda x: x['type'] == 'generated', axis=1)]))

Original interactions :  64
Generated interactions :  50
Number of different interactions :  22
Number of different interactions form generated :  4
